# Aquisição de dados no portal da ONS Dados Aberto

https://ons-aws-prod-opendata.s3.amazonaws.com/

https://registry.opendata.aws/ons-opendata-portal/

https://github.com/awslabs/open-data-registry/blob/main/datasets/ons-opendata-portal.yaml#L17

In [3]:
import sys
import os
import pandas as pd
import requests
import xml.etree.ElementTree as ET
from datetime import datetime,  timedelta

In [4]:
# Adiciona o caminho da pasta src ao sys.path para importar datasets_ons
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))
from datasets_ons import datasets_ons

In [5]:
def listar_arquivos():
    url = "https://ons-aws-prod-opendata.s3.amazonaws.com"
    params = {"list-type": "2", "prefix": "dataset/"}
    arquivos = []
    while True:
        resp = requests.get(url, params=params)
        root = ET.fromstring(resp.content)
        for item in root.findall(".//{*}Contents"):
            key_elem = item.find("{*}Key")
            if key_elem is not None and key_elem.text is not None:
                arquivos.append(key_elem.text)
        token = root.find(".//{*}NextContinuationToken")
        if token is not None and token.text is not None:
            params["continuation-token"] = token.text
        else:
            break
    return arquivos


def listar_arquivos_do_dataset(nome_dataset=None, extensoes=(".csv", ".parquet", ".xlsx", ".pdf")):
    todos = listar_arquivos()
    if nome_dataset is None:
        datasets = sorted({arq.split(
            "/")[1] for arq in todos if arq.startswith("dataset/") and not arq.endswith("/")})
        print(
            f"\nDatasets disponíveis ({datetime.now().strftime('%d/%m/%y')}):")
        for ds in datasets:
            print("-", ds)
        return

    arquivos = [arq.split("/")[-1] for arq in todos
                if arq.startswith(f"dataset/{nome_dataset}/") and not arq.endswith("/")
                and any(arq.endswith(ext) for ext in extensoes)]

    if not arquivos:
        print(f"Nenhum arquivo encontrado para o dataset '{nome_dataset}'.")
    else:
        print(f"\nArquivos do dataset '{nome_dataset}':")
        for arq in arquivos:
            print("-", arq)


def baixar_arquivo(nome_dataset, nome_arquivo, pasta_destino):
    url = f"https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/{nome_dataset}/{nome_arquivo}"
    os.makedirs(pasta_destino, exist_ok=True)
    caminho = os.path.join(pasta_destino, nome_arquivo)
    resp = requests.get(url)
    if resp.status_code == 200:
        with open(caminho, 'wb') as f:
            f.write(resp.content)
        print(f"Baixado: {nome_arquivo}")
    else:
        print(f"Erro {resp.status_code} ao baixar: {nome_arquivo}")


def baixar_dataset_ons(nome_dataset, pasta_destino, data_inicio=None, data_fim=None):
    """
    Baixa arquivos públicos do ONS a partir do dataset informado, com base na frequência dos dados
    (anual, mensal, diário ou fixo) definida no dicionário datasets_ons.

    Parâmetros:
    ----------
    nome_dataset : str
        Nome do dataset disponível no bucket ONS.
    pasta_destino : str
        Caminho onde os arquivos serão salvos localmente.
    data_inicio : str, opcional
        Data inicial no formato "YYYY", "YYYY-MM" ou "YYYY-MM-DD", dependendo da frequência do dataset.
        Obrigatória para datasets não fixos.
    data_fim : str, opcional
        Data final no formato "YYYY", "YYYY-MM" ou "YYYY-MM-DD", dependendo da frequência do dataset.
        Obrigatória para datasets não fixos.

    Retorna:
    -------
    None
        A função não retorna nada. Os arquivos são salvos localmente na pasta especificada.
    """
    if nome_dataset not in datasets_ons:
        print(f"Dataset '{nome_dataset}' não encontrado.")
        return

    info = datasets_ons[nome_dataset]
    padrao, freq = info['padrao'], info['frequencia']

    if freq == 'fixo':
        baixar_arquivo(nome_dataset, padrao, pasta_destino)
        return

    if not data_inicio or not data_fim:
        print(
            f"Datas de início e fim são obrigatórias para datasets do tipo '{freq}'.")
        return

    def parse_data(data):
        try:
            return datetime.strptime(data, "%Y-%m-%d")
        except ValueError:
            try:
                return datetime.strptime(data, "%Y-%m")
            except ValueError:
                return datetime.strptime(data, "%Y")

    dt_ini = parse_data(data_inicio)
    dt_fim = parse_data(data_fim)
    nomes = []

    if freq == 'anual':
        for ano in range(dt_ini.year, dt_fim.year + 1):
            nomes.append(padrao.format(ano=ano))

    elif freq == 'mensal':
        atual = datetime(dt_ini.year, dt_ini.month, 1)
        while atual <= dt_fim:
            nomes.append(padrao.format(ano=atual.year, mes=atual.month))
            ano = atual.year + (atual.month // 12)
            mes = atual.month % 12 + 1
            atual = datetime(ano, mes, 1)

    elif freq == 'diario':
        atual = dt_ini
        while atual <= dt_fim:
            nomes.append(padrao.format(ano=atual.year,
                         mes=atual.month, dia=atual.day))
            atual += timedelta(days=1)

    for nome_arq in nomes:
        baixar_arquivo(nome_dataset, nome_arq, pasta_destino)

In [6]:
baixar_dataset_ons("curva-carga-ho", pasta_destino="../data", data_inicio="2025-01-01", data_fim="2025-12-31")

Baixado: CURVA_CARGA_2025.csv


In [8]:
def pivotar_curva_carga_subsistemas(caminho_arquivo):
    """
    Lê um arquivo CSV da curva de carga por subsistema (no formato longo) e transforma para o formato largo,
    onde cada subsistema (Norte, Nordeste, Sul, Sudeste) se torna uma coluna e cada linha representa um timestamp.

    Parâmetros:
    ----------
    caminho_arquivo : str
        Caminho para o arquivo CSV de entrada, separado por ponto e vírgula.

    Retorno:
    -------
    df_largo : pandas.DataFrame
        DataFrame com uma linha por timestamp e colunas para cada subsistema.
    """
    df = pd.read_csv(caminho_arquivo, sep=";")

    df_largo = df.pivot(
        index="din_instante", columns="nom_subsistema", values="val_cargaenergiahomwmed")

    df_largo = df_largo.rename(columns={
        "NORTE": "Norte",
        "NORDESTE": "Nordeste",
        "SUL": "Sul",
        "SUDESTE": "Sudeste"
    })

    colunas_desejadas = ["Norte", "Nordeste", "Sul", "Sudeste"]
    colunas_presentes = [
        col for col in colunas_desejadas if col in df_largo.columns]
    df_largo = df_largo[colunas_presentes]

    df_largo = df_largo.reset_index()

    return df_largo

In [9]:
def preencher_horas_faltantes(df, metodo='spline', order=2):
    """
    Preenche as horas faltantes no índice datetime do DataFrame, interpolando os valores de acordo com o método escolhido.

    Parâmetros:
    df (pd.DataFrame): DataFrame com índice datetime e colunas numéricas.
    metodo (str): Método de interpolação ('linear', 'time', 'index', 'values', 'pad', 
                  'nearest', 'zero', 'slinear', 'quadratic', 'cubic', 'barycentric', 
                  'polynomial', 'krogh', 'piecewise_polynomial', 'spline', 'pchip', 
                  'akima', 'cubicspline', 'from_derivatives').
                  Padrão: 'spline'.
    order (int, opcional): Ordem do polinômio para os métodos 'polynomial' e 'spline'. Padrão: 2.

    Retorna:
    pd.DataFrame: DataFrame com as horas corrigidas e valores interpolados.
    """

    if df.empty:
        print("O DataFrame está vazio. Nenhuma interpolação foi realizada.")
        return df

    # Garante que o índice é datetime e remove valores inválidos
    df = df.copy()
    df.index = pd.to_datetime(df.index, errors='coerce')

    if df.index.isna().any():
        raise ValueError(
            "O índice contém valores não reconhecidos como datas.")

    # Criar um range de tempo contínuo com frequência de 1 hora
    full_range = pd.date_range(
        start=df.index.min(), end=df.index.max(), freq='h')

    # Só reindexa se houver horas ausentes
    if not df.index.equals(full_range):
        df = df.reindex(full_range)

    # Filtra apenas colunas numéricas
    numeric_cols = df.select_dtypes(include=['number']).columns
    non_numeric_cols = df.select_dtypes(exclude=['number']).columns

    if numeric_cols.empty:
        print("Nenhuma coluna numérica encontrada para interpolação. Retornando DataFrame original.")
        return df

    # Lista de métodos válidos
    metodos_validos = [
        'linear', 'time', 'index', 'values', 'pad', 'nearest', 'zero',
        'slinear', 'quadratic', 'cubic', 'barycentric', 'polynomial', 'krogh',
        'piecewise_polynomial', 'spline', 'pchip', 'akima', 'cubicspline', 'from_derivatives'
    ]

    # Aplicando o método de interpolação escolhido
    try:
        if metodo in metodos_validos:
            if metodo in ['spline', 'polynomial'] and order is None:
                order = 2  # Define um padrão se o usuário não passar
            df[numeric_cols] = df[numeric_cols].interpolate(
                method=metodo, order=order if metodo in ['spline', 'polynomial'] else None)
        else:
            raise ValueError(
                f"Método de interpolação inválido: '{metodo}'. Métodos disponíveis: {', '.join(metodos_validos)}")
    except Exception as e:
        print(f"Erro ao interpolar os dados: {e}")
        return df

    # Aviso se existirem colunas não numéricas
    if not non_numeric_cols.empty:
        print(
            f"As colunas não numéricas foram ignoradas: {list(non_numeric_cols)}")

    return df

In [10]:
def processar_pasta_curva_carga(pasta_entrada, salvar_em=None, metodo_interp='spline', ordem_interp=2):
    """
    Processa todos os arquivos CSV de curva de carga em uma pasta, transforma os dados para formato largo
    (pivotando os subsistemas) e concatena todos em um único DataFrame.

    Parâmetros:
    -----------
    pasta_entrada : str
        Caminho para a pasta contendo os arquivos CSV.
    salvar_em : str, opcional
        Caminho para salvar o arquivo consolidado. Se None, não salva.

    Retorno:
    --------
    df_final : pandas.DataFrame
        DataFrame unificado com os dados transformados de todos os arquivos.
    """
    arquivos_csv = sorted([
        os.path.join(pasta_entrada, f)
        for f in os.listdir(pasta_entrada)
        if f.endswith(".csv")
    ])

    dfs = []
    for caminho in arquivos_csv:
        try:
            df = pivotar_curva_carga_subsistemas(caminho)
            df['din_instante'] = pd.to_datetime(df['din_instante'])
            df.set_index('din_instante', inplace=True)
            print(f"Processando arquivo: {caminho}")
            df = preencher_horas_faltantes(
                df, metodo=metodo_interp, order=ordem_interp)
            print(f"Arquivo processado: {caminho} - {len(df)} linhas")
            if df.empty:
                print(f"Arquivo {caminho} está vazio após o processamento.")
                continue
            dfs.append(df)
        except Exception as e:
            print(f"Erro ao processar {caminho}: {e}")

    if not dfs:
        print("Nenhum arquivo válido encontrado na pasta.")
        return pd.DataFrame()

    df_final = pd.concat(dfs).sort_index().reset_index()

    if salvar_em:
        os.makedirs(os.path.dirname(salvar_em), exist_ok=True)
        df_final.to_csv(salvar_em, index=False)
        print(f"Arquivo consolidado salvo em: {salvar_em}")

    return df_final

In [11]:
curva2000 = processar_pasta_curva_carga(
    "../data/Curva Carga HO ONS",
    "../data/curva_carga_ho_2000_2025.csv")

Processando arquivo: ../data/Curva Carga HO ONS\CURVA_CARGA_2000.csv
Arquivo processado: ../data/Curva Carga HO ONS\CURVA_CARGA_2000.csv - 8784 linhas
Processando arquivo: ../data/Curva Carga HO ONS\CURVA_CARGA_2001.csv
Arquivo processado: ../data/Curva Carga HO ONS\CURVA_CARGA_2001.csv - 8760 linhas
Processando arquivo: ../data/Curva Carga HO ONS\CURVA_CARGA_2002.csv
Arquivo processado: ../data/Curva Carga HO ONS\CURVA_CARGA_2002.csv - 8760 linhas
Processando arquivo: ../data/Curva Carga HO ONS\CURVA_CARGA_2003.csv
Arquivo processado: ../data/Curva Carga HO ONS\CURVA_CARGA_2003.csv - 8760 linhas
Processando arquivo: ../data/Curva Carga HO ONS\CURVA_CARGA_2004.csv
Arquivo processado: ../data/Curva Carga HO ONS\CURVA_CARGA_2004.csv - 8784 linhas
Processando arquivo: ../data/Curva Carga HO ONS\CURVA_CARGA_2005.csv
Arquivo processado: ../data/Curva Carga HO ONS\CURVA_CARGA_2005.csv - 8760 linhas
Processando arquivo: ../data/Curva Carga HO ONS\CURVA_CARGA_2006.csv
Arquivo processado: ../da